In [1]:
import tensorflow as tf
import numpy as np
import os

2023-02-13 18:24:19.736942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 18:24:20.182440: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-13 18:24:20.236405: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-13 18:24:20.236420: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# Load the pretrained YOLOv3 model
model = tf.keras.models.load_model("model.h5")

2023-02-13 18:24:26.795029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-13 18:24:26.795288: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-13 18:24:26.795314: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ehsan-Swift-SF514-55T): /proc/driver/nvidia/version does not exist
2023-02-13 18:24:26.796168: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print([layer.name for layer in model.layers])
model.summary()

['dropout', 'dense', 'dropout_1', 'dense_1', 'dropout_2', 'dense_2']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 1000)              785000    
                                                                 
 dropout_1 (Dropout)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dropout_2 (Dropout)         (None, 1000)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                10010     
                                                     

In [4]:
# Specify the layers to be quantized
quantize_layer_names = ['dense_1']

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen


In [7]:
for layer_name in quantize_layer_names:
    # Convert the specified layers to int8
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    input_arrays = converter.get_input_arrays()
    for input_array in input_arrays:
        converter.quantizers[input_array].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.quantizers[layer_name].supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


AttributeError: 'TFLiteKerasModelConverterV2' object has no attribute 'get_input_arrays'

In [9]:
quantize_annotations = ['dense_1']

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.quantize_annotations = quantize_annotations
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7ow7d92n/assets


/home/ehsan/anaconda3/envs/Quantize/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-02-13 18:26:04.818615: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-13 18:26:04.818640: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-13 18:26:04.819419: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp7ow7d92n
2023-02-13 18:26:04.820501: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-13 18:26:04.820517: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp7ow7d92n
2023-02-13 18:26:04.824238: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimiza

In [11]:
open("test.tflite", "wb").write(quantized_tflite_model)

1804880